# Housing Sale Price Prediction : Advanced Regression Assignment

## Steps
1. Reading and Understanding the data
2. Missing Value Treatment
3. Data Analysis and Visualization
4. Preparing the data for Modelling
    - Dummy Encoding
    - Train - Test split
    - Rescaling
5. Training the Model
6. Regularization and Tuning
7. Residual Analysis
8. Predictions and Evaluations on the test set

### Step 1: Reading and understanding the data

In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Read the data
# Some of the columns have NA explicitely defined and it represents a particular category. So it shouldn't be treated as missing value. Hence reading them as string
cols_custom_read = ['Alley','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu','GarageType',\
                    'GarageFinish','GarageQual','GarageCond','PoolQC','Fence','MiscFeature']
custom_converters = {c : str for c in cols_custom_read}

housing_data = pd.read_csv('https://ml-course3-upgrad.s3.amazonaws.com/Assignment_+Advanced+Regression/train.csv', converters=custom_converters)
housing_data.head()

In [ ]:
# Understand the shape of the data
housing_data.shape

#### Observations
- Number of datapoints is 1460 which seems to be less
- There are 80 independent variables

In [ ]:
# Understand the data types, presence of missing values
housing_data.info()


### Step 2: Missing Value Treatment

In [ ]:
#List the columns which has missing values
housing_data_na = housing_data.isna().sum()
housing_data_na[housing_data_na > 0]

In [ ]:
# Imputing LotFrontage with median value
median_lot_frontage = housing_data['LotFrontage'].describe()['50%']

#Imputing MasVnrType with Mode. Since Missing values for MasVnrArea is exactly where MasVnrType is missing, we have to consider the value for Mode while imputing MasVnrArea
mode_mas_vnr_type = housing_data['MasVnrType'].mode()[0]

#Since Mode for MasVnrType is none, corresponding MasVnrArea should be 0
imputed_mas_vnr_area = 0

# No need to impute GarageYrBlt, because GarageYrBlt is missing for GarageType No Garage, It makes sense to keep it as it is.

#Impute Electrical with mode
mode_electrical = housing_data['Electrical'].mode()[0]

#Impute the dataframe with the above values
housing_data.fillna(value={'LotFrontage':median_lot_frontage,'MasVnrType':mode_mas_vnr_type,'MasVnrArea':imputed_mas_vnr_area,'Electrical':mode_electrical},inplace=True)

In [ ]:
#List the columns which has missing values after imputation
housing_data_na = housing_data.isna().sum()
housing_data_na[housing_data_na > 0]

In [ ]:
#Drop unneccessary columns
cols_to_be_dropped = ['Id']
housing_data.drop(cols_to_be_dropped,axis=1,inplace=True)

#Drop any duplicates
housing_data.drop_duplicates()
# There are no duplicated rows

In [ ]:
list(housing_data.columns)

### Step 3: Data Analysis and Visualization

In [ ]:
categorical_vars = ['MSSubClass','MSZoning','Street','Alley','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood','Condition1','Condition2',\
                    'BldgType','HouseStyle','OverallQual','OverallCond','YearBuilt','YearRemodAdd','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType',\
                    'ExterQual','ExterCond','Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Heating','HeatingQC','CentralAir', \
                    'Electrical','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','KitchenQual','TotRmsAbvGrd','Functional',\
                    'Fireplaces','FireplaceQu','GarageType','GarageYrBlt','GarageFinish','GarageCars','GarageQual','GarageCond','PavedDrive','PoolQC','Fence',\
                    'MiscFeature','MoSold','YrSold','SaleType','SaleCondition']
continuous_vars = ['LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea',\
                    'GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal']

In [ ]:
plt.figure(figsize=(20,10))
batch_size = 10
for i in range(0,len(continuous_vars),batch_size):
    sns.pairplot(data=housing_data,x_vars=continuous_vars[i:i+batch_size],y_vars='SalePrice')
plt.show()

#### Inferences
1. LotFrontage seems to have little influence on the SalePrice but seems to be linear influencer
2. LotArea seems to have not much influence on the target SalePrice
3. MasVnrArea, BsmtFinSF1 BsmtFinSF2, BsmtUnfSF seem to have little effect on the SalePrice. 
4. If we remove the value 0 for TotalBsmtSF,1stFlrSF, 2ndFlrSF, they seem to have to good influence on the target. 0 just represents that the particular category (basement, 1st floor or 2nd floor) is not there in the house
5. LowQualFinSF,WoodDeckSF,EnclosedPorch,3SsnPorch,ScreenPorch, PoolArea, MiscVal seems to have not much influence on the target, there are lot of zeros for these vars in the dataset which is also widely spread across different Sale Prices
6. GrLivArea and GarageArea seems to have a good linear influence on the SalePrice


In [ ]:
# Creating subplot axes
fig, axes = plt.subplots(12,5,figsize=(30,40), sharey=True)
for name, ax in zip(categorical_vars, axes.flatten()):
    sns.boxplot(y='SalePrice', x= name, data=housing_data, orient='v', ax=ax)
plt.show()


In [ ]:
# TODO : Write more inferences and find more columns to drop

#### Inferences
1. MSSubClass seems to be have some influence on the target variable.
2. Houses with Floating Village Residential & Residential Low Density MSZones tend to have higher Selling Price.
3. Houses with Paved road access tend to have higher SalePrice than Gravel
4. Houses with No Alley access tend to have higher SalePrice
5. Lotshape also have some influence on the target variable but not too much significant.
6. LandContour have some influence on the target variable. Higher SalePrice for Houses on Hillside and Depression
7. Utilities is heavily skewed towards AllPub. Just one datapoint for NoSeWa. Hence it is not an indicator for the SalePrice
8. Higher Influcence Predictors:
    - Neighbourhood
    - Condition1
    - Condition2
    - HouseStyle
    - OverAll quality
    - Overall Cond
    - Roof Material
    - Exterior Covering Materials (both 1 and 2)
    - Exterior Quality
    - Exterior Condition
    - Mass Vnr Type
    - 
9. Comparatively Lower Influence Predictors:
    - LotConfig
    - Landslope
    - Bldng type
    - Roof Style
7. YearBuilt and YearRemodel done doesn't seem to have good influence on the target ==>  Although more data is skewed towards the recent years, it has wide range of salePrice
8. Although recent values of GarageYrBlt have some high SalePrice, it is not a great influence for sale price
9. The variation of SalePrice w.r.t Month Sold or Year Sold is not significant enough



In [ ]:
# Drop columns based on EDA
cols_to_be_dropped = ['Utilities', 'LowQualFinSF','EnclosedPorch','3SsnPorch','ScreenPorch', 'PoolArea', 'MiscVal','MoSold','YrSold']
# cols_to_be_dropped.extend(['BsmtFinSF1','BsmtFinSF2','BsmtUnfSF'])
housing_data.drop(cols_to_be_dropped,axis=1,inplace=True)
print(f"Number of columns after dropping {housing_data.columns.size}")

In [ ]:
# Remove the dropped cols from continuous and categorical list
categorical_vars = [var for var in categorical_vars if var not in cols_to_be_dropped]
continuous_vars =  [var for var in continuous_vars if var not in cols_to_be_dropped]

### Step 4: Preparing the data for Modelling

#### Encoding

In [ ]:
# Dummy Encoding for all categorical variables
# Certain Numbers in Categorical Variable are not mapped to string , it is read as string to interpret it better. For eg: OverallCond_10 would be easier to interpret than OverallCond_Very_Excellent
dummy_encoded_values = pd.get_dummies(data=housing_data[categorical_vars].astype(str),drop_first=True)
categorical_vars_encoded = list(dummy_encoded_values.columns)

# Add the new encoded cols to original dataframe and drop the source columns
housing_data = pd.concat([housing_data,dummy_encoded_values],axis=1)
housing_data.drop(categorical_vars,axis=1,inplace=True)

In [ ]:
print(f"Number of columns after dummyEncoding {housing_data.columns.size}")

#### Train test split

In [ ]:
df_train, df_test = train_test_split(housing_data,train_size=0.7,random_state=100)
df_train.columns

#### Scaling the features using StandardScaler

In [ ]:
scaler = StandardScaler()
df_train[continuous_vars] = scaler.fit_transform(df_train[continuous_vars])
df_train[continuous_vars]

### Step 5: Model Builiding

#### Utilities

In [ ]:
# Calculate Adjusted R2
def adjusted_r2(X,r2_score):
    n = len(X)
    p = len(X.columns)
    return 1-(1-r2_score)*(n-1)/(n-p-1)

In [ ]:
# Add Constant
def add_constant(X_train):
    return sm.add_constant(X_train)

In [ ]:
## Build Stats Model and return summary
def build_stats_model(X_train,y_train):
    lm = sm.OLS(y_train,add_constant(X_train)).fit() # Fitting the model
    return lm.summary(),lm # Return summary

In [ ]:
# Compute VIF
def compute_vif(X_train):
    vif = pd.DataFrame()
    vif['Features'] = X_train.columns
    vif['VIF'] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2) # Rounding to 2 decimal values
    vif = vif.sort_values(by = 'VIF', ascending = False)
    return vif

In [ ]:
def get_model(model_type,alpha=1.0):
    if model_type == 'Ridge':
        return Ridge(alpha=alpha)
    if model_type == 'Lasso':
        return Lasso(alpha=alpha)

    raise ValueError(f"Invalid Model Type {model_type}") 
    

In [ ]:
def build_with_regularization(X_train,y_train,X_test,y_test,model_type,alpha_range,scoring='neg_root_mean_squared_error',folds=5):
    alpha_grid = [{'alpha': alpha_range}]
    grid_search_cv = GridSearchCV(get_model(model_type),param_grid= alpha_grid,scoring=scoring,cv=folds,n_jobs=-1,return_train_score=True)
    grid_search_cv.fit(X_train,y_train)
    best_alpha = grid_search_cv.best_params_['alpha']
    
    model = get_model(model_type,best_alpha)
    model.fit(X_train,y_train)

    y_train_pred = model.predict(X_train)
    train_r2_score = r2_score(y_true=y_train,y_pred=y_train_pred)
    y_test_pred = model.predict(X_test)
    test_r2_score =  r2_score(y_true=y_test,y_pred=y_test_pred)   
    return {'model': model, 'best_alpha': best_alpha, 'best_score': grid_search_cv.best_score_, 'train_r2_score': train_r2_score,'test_r2_score': test_r2_score}

#### Step 5.1: Model Building Without Regularization

In [ ]:
# Splitting the training data into X and y
y_train = df_train.pop('SalePrice')
X_train = df_train
y_test = df_test.pop('SalePrice')
X_test = df_test

In [91]:
# Create Linear Regression Model
lm = LinearRegression()
lm.fit(X_train,y_train)

# Running RFE with output number of values as 20
output_var_count = 50
rfe = RFE(lm,n_features_to_select=output_var_count)
rfe = rfe.fit(X_train, y_train)

In [92]:
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

[('LotFrontage', False, 439),
 ('LotArea', False, 364),
 ('MasVnrArea', False, 461),
 ('BsmtFinSF1', False, 227),
 ('BsmtFinSF2', False, 266),
 ('BsmtUnfSF', False, 267),
 ('TotalBsmtSF', False, 184),
 ('1stFlrSF', False, 408),
 ('2ndFlrSF', False, 515),
 ('GrLivArea', True, 1),
 ('GarageArea', False, 509),
 ('WoodDeckSF', False, 510),
 ('OpenPorchSF', False, 476),
 ('MSSubClass_160', False, 288),
 ('MSSubClass_180', False, 458),
 ('MSSubClass_190', False, 22),
 ('MSSubClass_20', False, 418),
 ('MSSubClass_30', False, 420),
 ('MSSubClass_40', False, 236),
 ('MSSubClass_45', False, 471),
 ('MSSubClass_50', False, 379),
 ('MSSubClass_60', False, 215),
 ('MSSubClass_70', False, 290),
 ('MSSubClass_75', False, 26),
 ('MSSubClass_80', False, 426),
 ('MSSubClass_85', False, 191),
 ('MSSubClass_90', False, 10),
 ('MSZoning_FV', False, 28),
 ('MSZoning_RH', False, 19),
 ('MSZoning_RL', True, 1),
 ('MSZoning_RM', False, 29),
 ('Street_Pave', False, 340),
 ('Alley_NA', False, 487),
 ('Alley_Pave

In [93]:
resulting_rfe_cols = X_train.columns[rfe.support_]
resulting_rfe_cols

Index(['GrLivArea', 'MSZoning_RL', 'Condition2_PosA', 'Condition2_PosN',
       'Condition2_RRAe', 'BldgType_Duplex', 'HouseStyle_2.5Unf',
       'OverallQual_10', 'OverallQual_2', 'OverallQual_3', 'OverallQual_4',
       'OverallQual_5', 'OverallQual_6', 'OverallQual_8', 'OverallQual_9',
       'YearBuilt_1890', 'YearBuilt_1893', 'YearBuilt_1898', 'YearBuilt_1919',
       'YearBuilt_1946', 'YearBuilt_1952', 'YearRemodAdd_1951',
       'YearRemodAdd_1952', 'RoofMatl_CompShg', 'RoofMatl_Membran',
       'RoofMatl_Metal', 'RoofMatl_Roll', 'RoofMatl_Tar&Grv',
       'RoofMatl_WdShake', 'RoofMatl_WdShngl', 'Exterior1st_CBlock',
       'Exterior2nd_CBlock', 'BsmtCond_NA', 'BsmtCond_Po', 'Heating_OthW',
       'KitchenAbvGr_1', 'KitchenAbvGr_2', 'KitchenAbvGr_3', 'TotRmsAbvGrd_14',
       'Functional_Sev', 'GarageType_NA', 'GarageYrBlt_1942.0',
       'GarageYrBlt_1946.0', 'GarageCars_1', 'GarageCars_3', 'GarageCars_4',
       'PoolQC_Gd', 'SaleType_Con', 'SaleType_New', 'SaleCondition_Parti

In [94]:
X_train.columns[~rfe.support_]

Index(['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GarageArea',
       ...
       'SaleType_CWD', 'SaleType_ConLD', 'SaleType_ConLI', 'SaleType_ConLw',
       'SaleType_Oth', 'SaleType_WD', 'SaleCondition_AdjLand',
       'SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal'],
      dtype='object', length=522)

In [95]:
# Keeping only the columns from RFE
X_train_rfe = X_train[resulting_rfe_cols]
X_test_rfe = X_test[resulting_rfe_cols]

In [96]:
# Create Linear Regression Model
lm = LinearRegression()
lm.fit(X_train_rfe,y_train)
y_train_pred = lm.predict(X_train_rfe)
y_test_pred = lm.predict(X_test_rfe)
train_r2_score  = r2_score(y_true=y_train,y_pred=y_train_pred)
train_adj_r2_scopre = adjusted_r2(X_train_rfe,train_r2_score)
test_r2_score  = r2_score(y_true=y_test,y_pred=y_test_pred)
test_adj_r2_score = adjusted_r2(X_test_rfe,test_r2_score)
print(f"R2 score of training {train_r2_score}")
print(f"Adj R2 score of training {train_adj_r2_scopre}")
print(f"R2 score of test {test_r2_score}")
print(f"Adj R2 score of test {test_adj_r2_score}")

R2 score of training 0.8681052501406288
Adj R2 score of training 0.8613065516942694
R2 score of test -457121.0679857954
Adj R2 score of test -516028.55097365566


##### Inference
- Even though R2 and Adj R2 score for training set seems to be decent , it miserably failed for test data. Let's look at the statistics to understand the significance of the variables

In [97]:
# Get the List of Independent Variables with high p value and/or high vif:
def get_correlated_unreliable_ind_vars(X_train,y_train):
    # Build stats model and get summary
    (results_summary,_) = build_stats_model(X_train,y_train)

    #Read the Metrics from OLS Summary
    results_as_html_0 = results_summary.tables[0].as_html()
    summary_df_0 = pd.read_html(results_as_html_0,index_col=2)[0][[3]]
    summary_df_0.index.name = 'Metric'
    summary_df_0.rename({3:'Value'},axis=1,inplace=True)

    # Read the p value table from OLS Summary
    results_as_html_1 = results_summary.tables[1].as_html()
    summary_df_1 = pd.read_html(results_as_html_1, header=0, index_col=0)[0]
    summary_filtered_df = summary_df_1[summary_df_1['P>|t|'] > 0.05].sort_values('P>|t|',ascending=False)


    #Compute VIF
    vif = compute_vif(X_train)
    vif_filtered_df = vif[vif['VIF'] > 5]

    #Merge both df to combine variables which needs attention
    corr_unr_ind_vars = summary_filtered_df.merge(vif_filtered_df,how='outer',left_index=True,right_on=['Features'])[['Features','P>|t|','VIF']]
    corr_unr_ind_vars.reset_index(drop=True,inplace=True)
    corr_unr_ind_vars.sort_values(by=['P>|t|','VIF'],ascending=False,inplace=True)
    return (corr_unr_ind_vars,summary_df_0,summary_df_1,results_summary,vif)
    

In [98]:
#Recursively remove features based on stats model summary statistics
def recursive_feature_removal(X_train,y_train,iter=1):
    result = []
    (corr_unr_ind_vars,summary_df_0,_,_,vif) = get_correlated_unreliable_ind_vars(X_train,y_train)
   
    if corr_unr_ind_vars.empty:
        top_row = {'Features': None,'VIF': None,'P>|t|': None}
    else:
        top_row = corr_unr_ind_vars.iloc[0]
        
    var_to_drop = top_row['Features']
    vif = top_row['VIF']
    p_value = top_row['P>|t|']
    r2 = summary_df_0.iloc[0]['Value']
    adj_r2 = summary_df_0.iloc[1]['Value']
    result.append({
        'iter': iter,
        'var_to_drop': var_to_drop,
        'vif': vif,
        'p_value': p_value,
        'r2_score': r2,
        'adj_r2_score': adj_r2,
        'X_train': X_train
    })
    if not corr_unr_ind_vars.empty:
        result.extend(recursive_feature_removal(X_train.drop([var_to_drop],axis=1),y_train,iter+1))
    return result


In [99]:
rfr_result = recursive_feature_removal(X_train_rfe,y_train)
print(rfr_result)

[{'iter': 1, 'var_to_drop': 'YearBuilt_1898', 'vif': inf, 'p_value': 0.8270000000000001, 'r2_score': 0.868, 'adj_r2_score': 0.862, 'X_train':       GrLivArea  MSZoning_RL  Condition2_PosA  Condition2_PosN  \
318    2.121655            1                0                0   
239   -0.058599            1                0                0   
986    0.219811            0                0                0   
1416   1.451965            0                0                0   
390   -0.284338            1                0                0   
...         ...          ...              ...              ...   
802   -0.116915            1                0                0   
53     0.609209            1                0                0   
350    0.660000            1                0                0   
79    -0.542055            0                0                0   
792    0.930886            1                0                0   

      Condition2_RRAe  BldgType_Duplex  HouseStyle_2.5Unf  Overal

In [100]:
X_train_rfe = rfr_result[-1]['X_train']
X_test_rfe = X_test_rfe[X_train_rfe.columns]

#### Step 5.2: Model Building With Ridge

In [101]:
print(f"Building Model Using Ridge ")
ridge_alphas = list(np.linspace(0,1,100))
result = build_with_regularization(X_train_rfe,y_train,X_test_rfe,y_test,'Ridge',ridge_alphas)
print(f"Best Alpha: {result['best_alpha']}")
print(f"Best Score: {result['best_score']}")
print(f"Training R2 Score: {result['train_r2_score']}")
print(f"Test R2 Score: {result['test_r2_score']}")

Building Model Using Ridge 
Best Alpha: 0.8989898989898991
Best Score: -36780.63030927898
Training R2 Score: 0.8399202989711283
Test R2 Score: -298580.5703329977


In [102]:
X_train_rfe.columns.size

26

#### Step 5.3: Model Building With Lasso

In [103]:
print(f"Building Model Using Lasso ")
lasso_alphas = list(np.arange(0,1000,1))
result = build_with_regularization(X_train_rfe,y_train,X_test_rfe,y_test,'Lasso',lasso_alphas)
print(f"Best Alpha: {result['best_alpha']}")
print(f"Best Score: {result['best_score']}")
print(f"Training R2 Score: {result['train_r2_score']}")
print(f"Test R2 Score: {result['test_r2_score']}")

Building Model Using Lasso 


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: UserWarning: Coor

Best Alpha: 164
Best Score: -37210.551331716
Training R2 Score: 0.833811780562157
Test R2 Score: -320989.66072608327
